In [28]:
import uproot as up
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-v0_8-paper')
plt.rcParams["patch.force_edgecolor"] = True

In [29]:
file_features = "../../../../mnt/fcufino/all_features_IBD_He.csv"
# file_features = "../../../../mnt/fcufino/all_features_IBD_U.csv"
# file_features = "../../../../mnt/fcufino/all_features_IBD_Li.csv"
# file_features = "../../../../mnt/fcufino/all_features_IBD_Th.csv"



In [30]:
all_feature_df = pd.read_csv(file_features)
all_feature = {col: all_feature_df[col].values for col in all_feature_df.columns}



features_BKG_df = all_feature_df[all_feature_df['Label'] == 0] #Qui c'è sia IBD_fake che BKG_radioactive
features_BKG = {col: features_BKG_df[col].values for col in features_BKG_df.columns}

features_IBD_df = all_feature_df[all_feature_df['Label'] == 1] #Label = 1 sono i VERI ibd
features_IBD = {col: features_IBD_df[col].values for col in features_IBD_df.columns}


# Cutting algorithm for BOTH IBD and BKG

In [31]:
from numba import njit, prange

In [32]:
@njit(parallel = True) 
#delta time in nanoseconds
#delta radius in mm
def selection(dt,dr,E_pro,E_del, delta_time = 1e6, delta_radius = 1500, min_energy_prompt = 0.7,max_energy_prompt = 12, min_energy_delay = 1.9, max_energy_delay = 2.5, min_energy_delay_carb = 4.4, max_energy_delay_carb = 5.5):
    prompt_columns = np.zeros(dt.shape)
    delay_columns = np.zeros(dt.shape)
    delay_columns_carb = np.zeros(dt.shape)
    for i in prange(dt.shape[0]):
        if dt[i] < delta_time: 
            if dr[i] < delta_radius: 
                if E_pro[i]>= min_energy_prompt and E_pro[i]<= max_energy_prompt:
                    if E_del[i]>= min_energy_delay and E_del[i]<= max_energy_delay:
                        prompt_columns[i] = 1
                        delay_columns[i] = 1
                    if E_del[i]>= min_energy_delay_carb and E_del[i]<= max_energy_delay_carb:
                        prompt_columns[i] = 1
                        delay_columns_carb[i] = 1
    return prompt_columns,delay_columns,delay_columns_carb

### IBD selection, searching for true IBD events

In [33]:
prompt_columns_IBD, delay_columns_IBD, delay_columns_carb_IBD = selection(features_IBD["delta_time"],
                                                                           features_IBD["delta_radius"],
                                                                           features_IBD["E_pro"],
                                                                           features_IBD["E_del"])

In [34]:
print("Venongono selezionati come IBD ", prompt_columns_IBD.sum(),"Su un tot di", prompt_columns_IBD.shape[0])


TP = np.logical_and(features_IBD['Label'] == 1, prompt_columns_IBD == 1)
TN = np.logical_and(features_IBD['Label'] == 0, prompt_columns_IBD == 0)
FN = np.logical_and(features_IBD["Label"] == 1, prompt_columns_IBD == 0)
FP = np.logical_and(features_IBD["Label"] == 0, prompt_columns_IBD == 1) 

print("Efficiency: ", (TP.sum()/(TP.sum()+FN.sum())) * 100)
#Efficiency sono il numero di eventi IBD veri selezionati diviso il numero totale di IBD.
print("Purity: ", (TP.sum()/(TP.sum()+FP.sum())) * 100)	

Venongono selezionati come IBD  48165.0 Su un tot di 49277
Efficiency:  97.74336911743815
Purity:  100.0


### BKG selection, searching for fake IBD eventi in BKG_RADIOACTIVITY dataset

In [35]:
prompt_columns_BKG, delay_columns_BKG, delay_columns_carb_BKG = selection(features_BKG["delta_time"][features_BKG["Source"] == 0],
                                                                           features_BKG["delta_radius"][features_BKG["Source"] == 0],
                                                                           features_BKG["E_pro"][features_BKG["Source"] == 0],
                                                                           features_BKG["E_del"][features_BKG["Source"] == 0])

In [36]:
print(prompt_columns_BKG.sum(), prompt_columns_BKG.shape[0])

TP = np.logical_and(features_BKG['Label'][features_BKG["Source"] == 0] == 0, prompt_columns_BKG == 0)
TN = np.logical_and(features_BKG['Label'][features_BKG["Source"] == 0] == 1, prompt_columns_BKG == 1)
FN = np.logical_and(features_BKG["Label"][features_BKG["Source"] == 0] == 0, prompt_columns_BKG == 1)
FP = np.logical_and(features_BKG["Label"][features_BKG["Source"] == 0] == 1, prompt_columns_BKG == 0) 

print("Efficiency: ", (TP.sum()/(TP.sum()+FN.sum())) * 100)
print("Purity: ", (TP.sum()/(TP.sum()+FP.sum())) * 100)	

43683.0 44916
Efficiency:  2.745124231899546
Purity:  100.0


# TOTAL Cut Algorithm 

In [37]:
prompt_columns, delay_columns, delay_columns_carb = selection(all_feature["delta_time"],
                                                              all_feature["delta_radius"],
                                                              all_feature["E_pro"],
                                                              all_feature["E_del"])

In [38]:
TP = np.logical_and(all_feature['Label'] == 1, prompt_columns == 1)
TN = np.logical_and(all_feature['Label'] == 0, prompt_columns == 0)
FN = np.logical_and(all_feature["Label"] == 1, prompt_columns == 0)
FP = np.logical_and(all_feature["Label"] == 0, prompt_columns == 1) 

print("Efficiency: ", (TP.sum()/(TP.sum()+FN.sum())) * 100)
print("Purity: ", (TP.sum()/(TP.sum()+FP.sum())) * 100)	

Efficiency:  97.74336911743815
Purity:  52.43990070551345
